In [1]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torch
from torch import optim, nn
from torchvision import models, transforms
from google.colab import drive
import pandas as pd
import numpy as np
import tensorflow as tf
import os
#from keras.models import Sequential
#from keras.layers import Convolution2D, MaxPooling2d, Flatten, Dense, Dropout, GlobalAveragePooling2D
#from keras.applications import VGG16
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def process_image(model, image):

  preprocess = weights.transforms()

  batch = preprocess(image).unsqueeze(0)

  prediction = model(batch).squeeze(0).softmax(0)
  class_id = prediction.argmax().item()
  score = prediction[class_id].item()
  category_name = weights.meta["categories"][class_id]
  print(f"{category_name}: {100 * score:.2f}%")

directory_name = r'/content/drive/MyDrive/CS482/coco_minitrain_25k/images/train2017/'
dir = os.listdir(directory_name)


#https://pytorch.org/vision/stable/models.html >> USED THIS <<
#pretrained model weights (currently alias for IMAGENET1K_V2)
weights = ResNet50_Weights.DEFAULT
#model using pretrained model weights
model = resnet50(weights=weights)
# Set model to eval mode
model.eval()
count = 0
for img in dir:
  if count > 10:
    break
# https://pytorch.org/vision/stable/generated/torchvision.io.read_image.html >> USED THIS <<
  process_image(model, read_image(directory_name + img))
  count += 1
  
knn_index = {
    "settings": {
        "index.knn": True
    },
    "mappings": {
        "properties": {
            "zalando_img_vector": {
                "type": "knn_vector",
                "dimension": 2048
            }
        }
    }
}

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

house finch: 36.60%
traffic light: 39.56%
tabby: 31.27%
lab coat: 6.75%
thatch: 33.56%
bell cote: 40.84%
toilet tissue: 9.16%
umbrella: 48.28%
croquet ball: 6.82%
leopard: 40.52%
banana: 50.41%


In [4]:
#Feature Extraction Class

class featureExtractor(nn.Module):
  def __init__(self,model):
    super(featureExtractor, self).__init__()
    self.features = list(model.features)
    self.features = nn.Sequential(*self.features)
    self.pooling = model.avgpool
    self.flatten = nn.Flatten()
    self.fc = model.classifier[0]
  def forward(self, x):
      out = self.features(x)
      out = self.pooling(out)
      out = self.flatten(out)
      out = self.fc(out) 
      return out 
model = models.vgg16(pretrained=True)
new_model = featureExtractor(model)

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
#Iterating through directory and extracting features from each image
from tqdm import tqdm
import cv2
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(512),
    transforms.Resize(448),
    transforms.ToTensor()
])

features = []
for img in dir:
  img_path = directory_name + img
  image = cv2.imread(img_path)
  image = transform(image)
  image = image.reshape(1,3,448,448)
  image = image.to(device)
  with torch.no_grad():
    feature = new_model(image)
  features.append(feature.cpu().detach().numpy().reshape(-1))
features = np.array(features)

In [ ]:
#printing out what labels were given to each image from directory we went over
from sklearn.cluster import KMeans
model = KMeans(n_clusters = 5, random_state = 42)
model.fit(features)
labels = model.labels_
print(labels)